<a href="https://colab.research.google.com/github/bdemchak/cytoscape-jupyter/blob/main/gangsu/basic%20protocol%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a work-in-progress reproduction of the [Biological Network Exploration with Cytoscape 3](https://pubmed.ncbi.nlm.nih.gov/25199793/) Basic Protocol 1, which loads an s. cervesiae network, filters out unneeded nodes, lays out the resulting network, and then creates a dendogram display.

While much of it works, there are compromises, mainly due to Cytoscape features that aren't at full strength yet.

---
#Setup data files, py4cytoscape and Cytoscape connection
---
**NOTE: To run this notebook, you must manually start Cytoscape first -- don't proceed until you have started Cytoscape.**

---
##Setup: Notebook data files

Create the 'output' directory, which will be used to store files uploaded from Cytoscape.

This is a good place to prepare any other system resources that might be needed by downstream Notebook cells.





In [1]:
!rm -r output/
!ls -l 
OUTPUT_DIR = 'output/'

rm: cannot remove 'output/': No such file or directory
total 4
drwxr-xr-x 1 root root 4096 Mar  5 14:37 sample_data


---
##Setup: Fetch latest py4cytoscape




**Note that you can fetch from a specific github branch by adding "@<branch>" to the "py4cytocape" at the end of the github URL.**

For example, to get branch 0.0.5: git+https://github.com/cytoscape/py4cytoscape@0.0.5

In [2]:
!pip uninstall -y py4cytoscape

#!pip install py4cytoscape
!pip install git+https://github.com/cytoscape/py4cytoscape@0.0.8
#!pip install git+https://github.com/cytoscape/py4cytoscape

import py4cytoscape as p4c

  Cloning https://github.com/cytoscape/py4cytoscape (to revision 0.0.8) to /tmp/pip-req-build-xt77zx9j
  Running command git clone -q https://github.com/cytoscape/py4cytoscape /tmp/pip-req-build-xt77zx9j
  Running command git checkout -b 0.0.8 --track origin/0.0.8
  Switched to a new branch '0.0.8'
  Branch '0.0.8' set up to track remote branch '0.0.8' from 'origin'.
     |████████████████████████████████| 3.1MB 4.5MB/s 
  Created wheel for py4cytoscape: filename=py4cytoscape-0.0.8-cp37-none-any.whl size=144053 sha256=faa6abc38eb0b90008160eae8280ec40aee60525b32519af586ed9f430fce2a2
  Stored in directory: /tmp/pip-ephem-wheel-cache-px64xnep/wheels/50/fb/ad/2ef86b83249494e3b5793a114c7b3640f4c5f926fbfc9c23c8
Successfully built py4cytoscape


---
##Setup: Set up Cytoscape connection


Set up a "browser client", which is the glue that allows the server-based Notebook to communicate with Cytoscape.

*Note that the IPython.display.Javascript() call must be the last line in this cell.*

In [3]:
import IPython

print(f'Loading Javascript client ... {p4c.get_browser_client_channel()} on {p4c.get_jupyter_bridge_url()}')
browser_client_js = p4c.get_browser_client_js(True)
IPython.display.Javascript(browser_client_js) # Start browser client


Loading Javascript client ... 634b76fe-240c-4d71-85ef-8f30ad26d2aa on https://jupyter-bridge.cytoscape.org


<IPython.core.display.Javascript object>

---
#Sanity test to verify Cytoscape connection


By now, the connection to Cytoscape should be up and available. To verify this, try a simple operation that doesn't alter the state of Cytoscape.

In [37]:
p4c.cytoscape_version_info()


{'apiVersion': 'v1',
 'automationAPIVersion': '1.0.0',
 'cytoscapeVersion': '3.9.0-SNAPSHOT',
 'jupyterBridgeVersion': '0.0.2',
 'py4cytoscapeVersion': '0.0.8'}

---
##Setup: Import source data files

The network and annotation files are in a Dropbox folder, and this cell downloads them into the default Sandbox from where Cytoscape will access them.

The files could just as well have been on any cloud resource, including Google Drive, Github, Microsoft OneDrive or a private web site. Note that in this case, the network file was so large that it could not be saved on GitHub, so Dropbox was a handy alternative.

*An alternative would be to load the files into this Notebook file system (or create them there) and then download those files to the Sandbox. Loading them into the Notebook file system would require the use of Notebook "!" commands (e.g., !wget).*

In this cell, we show how the network file can be directly downloaded from Dropbox. The cell that demonstrates gene expression merging (below) shows the use of the "!" command.

**Sandboxing is explained in https://py4cytoscape.readthedocs.io/en/latest/concepts.html#sandboxing**

In [38]:
p4c.sandbox_set(None) # Revert to default sandbox in case some other workflow selected a different one

res_mitab = p4c.sandbox_url_to("https://www.dropbox.com/s/8wc8o897tsxewt1/BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab?dl=0", "BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab")
print(f'Network file BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab has {res_mitab["fileByteCount"]} bytes')

Network file BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab has 166981992 bytes


# Load the s. cerevisiae MITAB network into Cytoscape

Note that the import_network_from_file function (incorrectly) throws an exception, so we explicitly ignore the exception.

**Note**: Once CYTOSCAPE-12772 is fixed, we can remove the try-block in this cell.

In [56]:
from requests import HTTPError
p4c.close_session(False)

try:
  p4c.import_network_from_file('BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab')
except:  
  pass
if p4c.get_network_count() != 1:
  raise Exception('Failed to load network')
net_suid = p4c.get_network_suid()
net_suid



In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/C:/Users/CyDeveloper/CytoscapeConfiguration/3/framework-cytoscape.log'}


124

# Merge the gene expression data into the node table

For Cytoscape 3.9.0 and later, call Cytoscape to merge the gene expression data into the node attribute table. 

For pre-Cytoscape 3.9.0, do most of the work in Pandas and then import the dataframe into the node attribute table. Explicitly set the Gene ID as a string even though it's originally parsed as a number. To Cytoscape, the string will be comparable to the 'name' column already in the BIOGRID network. The Gene ID column in the dataframe is matched to the network's name column.



In [57]:
if p4c.check_supported_versions(cytoscape='3.9') is None:
  # Load file directly into Sandbox so Cytoscape can import it
  res_soft = p4c.sandbox_url_to("https://www.dropbox.com/s/r15azh0xb53smu1/GDS112_full.soft?dl=0", "GDS112_full.soft")
  print(f'Annotation file GDS112_full.soft has {res_soft["fileByteCount"]} bytes')

  res = p4c.load_table_data_from_file('GDS112_full.soft', start_load_row=83, data_key_column_index=10)
  print(res)
else:
  # Load file into Notebook file system so Python can import it, tweak it, and download to Cytoscape
  !wget -q --no-check-certificate https://www.dropbox.com/s/r15azh0xb53smu1/GDS112_full.soft?dl=0
  !mv GDS112_full.soft?dl=0 GDS112_full.soft

  import pandas as df
  GDS112_full = df.read_csv('GDS112_full.soft', skiprows=82, sep='\t')
  GDS112_full.dropna(subset=['Gene ID'], inplace=True)
  GDS112_full['Gene ID'] = df.to_numeric(GDS112_full['Gene ID'], downcast='integer')
  GDS112_full = GDS112_full.astype({'Gene ID': 'string'})
  print(GDS112_full.dtypes)
  print(GDS112_full)
  p4c.load_table_data(GDS112_full, data_key_column='Gene ID')

  import os
  os.remove('GDS112_full.soft')


Annotation file GDS112_full.soft has 5536880 bytes
{'mappedTables': [95, 133]}


# Create a filter to remove nodes having no Gene Symbol

In [58]:
p4c.create_column_filter('SymbolOK', 'Gene symbol', '[A-Z0-9]*', 'REGEX')


No edges selected.


{'edges': None,
 'nodes': ['852676',
  '851657',
  '854688',
  '853970',
  '851846',
  '854262',
  '851146',
  '851034',
  '852090',
  '853654',
  '856886',
  '855010',
  '856788',
  '855122',
  '856453',
  '855048',
  '856568',
  '856887',
  '855706',
  '850521',
  '855463',
  '856203',
  '852732',
  '852715',
  '854866',
  '852893',
  '853737',
  '853016',
  '853939',
  '854809',
  '855339',
  '851259',
  '856530',
  '854327',
  '855998',
  '853304',
  '854980',
  '852564',
  '853566',
  '856105',
  '853868',
  '856858',
  '853861',
  '855344',
  '851017',
  '852168',
  '855488',
  '856765',
  '852979',
  '851136',
  '854020',
  '854223',
  '854290',
  '852582',
  '850923',
  '854933',
  '851230',
  '850364',
  '850545',
  '852607',
  '852665',
  '856417',
  '856911',
  '854776',
  '851150',
  '856290',
  '854227',
  '850324',
  '852865',
  '854204',
  '851213',
  '852126',
  '853666',
  '850544',
  '856201',
  '852382',
  '850418',
  '851318',
  '851968',
  '854241',
  '855253',
  '

# Create a subnetwork containing only named nodes

This could take several minutes.

At the end, you should see a view containing all nodes laid out. 

If you see only a single rectangle, it could be that your Cytoscape is set to operate with a small stack size. To increase the stack:

1. terminate Cytoscape
2. use a text editor to add -Xss4096k to the cytoscape.vmoptions file in your Cytoscape program directory
4. restart Cytoscape
5. re-run this workflow

In [59]:
new_suid = p4c.create_subnetwork()
new_suid

1387937

# Get rid of the original network, which isn't needed anymore

In [10]:
p4c.delete_network(net_suid)
net_suid = new_suid

# Install clusterMaker2 if it hasn't already been installed

In [ ]:
p4c.install_app('clusterMaker2')

{}


{}

# Create the hierarchical clustering and dendogram

This returns a large data structure that describes the dendogram.

It also creates a dendogram window that's designed for GUI manipulation. It's unclear this can be controlled or used by automation calls.

**Note:** Having the dendogram is important, and so is having the data that created it. When CSD-420 is addressed, it will be possible to snapshot the dendogram and perform other operations with it.

In [ ]:
p4c.commands_post('cluster hierarchical showUI=true clusterAttributes=false nodeAttributeList="GSM1029,GSM1030,GSM1032,GSM1033,GSM1034"')

[{'nodeOrder': [{'nodeName': '851820', 'suid': 2915215},
   {'nodeName': '851924', 'suid': 2772850},
   {'nodeName': '854487', 'suid': 2770888},
   {'nodeName': '853985', 'suid': 2774704},
   {'nodeName': '854157', 'suid': 2784205},
   {'nodeName': '851208', 'suid': 2771047},
   {'nodeName': '852107', 'suid': 2775967},
   {'nodeName': '852854', 'suid': 3105367},
   {'nodeName': '850695', 'suid': 2773087},
   {'nodeName': '851125', 'suid': 2772907},
   {'nodeName': '850422', 'suid': 2772916},
   {'nodeName': '852549', 'suid': 2778433},
   {'nodeName': '851366', 'suid': 2826481},
   {'nodeName': '854469', 'suid': 2842171},
   {'nodeName': '852704', 'suid': 2783527},
   {'nodeName': '853534', 'suid': 2792362},
   {'nodeName': '850820', 'suid': 2772909},
   {'nodeName': '855473', 'suid': 2774098},
   {'nodeName': '853762', 'suid': 2783584},
   {'nodeName': '853682', 'suid': 2771029},
   {'nodeName': '850894', 'suid': 2773231},
   {'nodeName': '856063', 'suid': 2771077},
   {'nodeName': '85

#Use BiNGO for enrichment analysis

The BiNGO app doesn't have automation entrypoints, so this analysis isn't possible right now. Is there a different app that can do this?

**NOTE:** We need CSD-421 fixed because we don't have any analysis right now, which is very important.